In [1]:
suppressMessages({
    library(ArchR)
    library(anndata)
    library(tidyverse)
})

# Define custom functions for GRanges conversion
GRangesToString <- function(grange, sep = c(":", "-")) {
  regions <- paste0(
    as.character(seqnames(grange)),
    sep[[1]],
    start(grange),
    sep[[2]],
    end(grange)
  )
  return(regions)
}

StringToGRanges <- function(regions, sep = c(":", "-"), ...) {
  ranges.df <- data.frame(ranges = regions)
  ranges.df <- separate(
    data = ranges.df,
    col = "ranges",
    sep = paste0(sep[[1]], "|", sep[[2]]),
    into = c("chr", "start", "end")
  )
  granges <- makeGRangesFromDataFrame(df = ranges.df, ...)
  return(granges)
}

In [3]:
atac_path <- "/data1/chanj3/HTA.multiome.lung.NE_plasticity.060124/results/atac_peak2gene/RU151_PDX3_peak2gene"
rna_path <- "/data1/chanj3/HTA.multiome.lung.NE_plasticity.060124/results/atac_network_analysis/RU151_PDX3/manual_annotation/RU151_PDX3_rna_manual_annotation.h5ad"

proj <- loadArchRProject(path = atac_path)
rna_adata <- read_h5ad(rna_path)
atac_cells = map_chr(proj$cellNames, ~strsplit(.x, split="#")[[1]][2])

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [ ]:
# take the intersection of the atac and rna data
intersecting_cells = intersect(atac_cells, rownames(rna_adata))
rna_adata = rna_adata[intersecting_cells,]
proj = proj[paste0("RU151_PDX3#", intersecting_cells),]
cell_type_labels = rna_adata$obs['cell_type']
proj$cell_type <- cell_type_labels$cell_type

In [ ]:
library(reticulate)
pd <- import("pandas")
annot_fn = file.path('/data1/chanj3/HTA.multiome.lung.NE_plasticity.060124/results/atac_insilicochip/RU151_PDX3_tf_peaks.ISChIP_idx.pkl')
tf_peaks_idx <- pd$read_pickle(annot_fn)
tf_peaks_df = read.table('/data1/chanj3/HTA.multiome.lung.NE_plasticity.060124/results/atac_peak2gene/RU151_PDX3_peak2gene/peak_counts/peaks.csv',
                         sep=',', header=T)
tf_peaks <- list()
for (tf in names(tf_peaks_idx)) {
    ind = tf_peaks_idx[[tf]] + 1
    select_df = tf_peaks_df[ind,]
    tf_peaks[[tf]] <- GRanges(select_df$seqnames, IRanges(select_df$start, select_df$end))
}


In [39]:
peaks_phox2b = GRangesToString(tf_peaks[['PHOX2B']])

In [51]:
p2g = getPeak2GeneLinks(proj, 
    corCutOff = 0.1,
    FDRCutOff = 1,
    resolution = 1,
    returnLoops = FALSE
)
p2g$geneName <- mcols(metadata(p2g)$geneSet)$name[p2g$idxRNA]
p2g$peakName <- (metadata(p2g)$peakSet %>% {paste0(seqnames(.), ":", start(.), "-", end(.))})[p2g$idxATAC]

In [53]:
phox2b_targets = sort(unique(p2g[p2g$peakName %in% peaks_phox2b,'geneName']))

In [57]:
deg_df = read.csv("/data1/chanj3/HTA.multiome.lung.NE_plasticity.060124/results/atac_network_analysis/RU151_PDX3/manual_annotation/RU151_PDX3_NE_high_markers.csv",
                   row.names = 1)

In [58]:
deg_df

,feature.name,log2.fold_change.,p.value,adjusted.p.value
,<chr>,<dbl>,<dbl>,<dbl>
0,chr19:37251654-37252155,3.665316,8.682953e-62,6.197892e-58
1,chr19:20052037-20052538,6.499756,2.453504e-50,8.756556e-47
2,chr19:20661432-20661933,6.677804,2.173671e-41,5.171888e-38
3,chr19:20039313-20039814,5.195481,4.989060e-38,8.902977e-35
4,chr19:36331541-36332042,2.152154,1.003595e-36,1.432733e-33
5,chr19:22052173-22052674,5.387914,9.501604e-31,1.130374e-27
6,chr12:125983415-125983916,4.827750,1.559553e-30,1.590299e-27
7,chr19:21569065-21569566,4.577392,2.777771e-29,2.478466e-26
8,chr19:20424742-20425243,4.730403,1.262225e-28,1.001085e-25
